In [ ]:
import os
import cv2
import random
from random_word import RandomWords
import numpy as np
from pascal_voc_writer import Writer
from numpy import sin, cos, sqrt, pi
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.2 MB/s eta 0:00:00


ModuleNotFoundError: ignored

In [ ]:
def add_watermark(image1, counter):
    image = image1
    r = RandomWords()
    check = True
    while(check):
        text = ''
        for i in range(random.randint(1, 3)):
            text = text + r.get_random_word()
        fontScale = random.uniform(0.7, 1.25)
        opacity = random.uniform(0.5, 1.0)
        rotation = random.randint(-45, 45)
        if(fontScale >= 1):
            thickness = random.randint(1,2)
        else:
            thickness = 1
        font = random.choice([cv2.FONT_HERSHEY_SIMPLEX, cv2.FONT_HERSHEY_PLAIN, cv2.FONT_HERSHEY_DUPLEX, cv2.FONT_HERSHEY_COMPLEX, cv2.FONT_HERSHEY_TRIPLEX, cv2.FONT_HERSHEY_COMPLEX_SMALL, cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, cv2.FONT_HERSHEY_SCRIPT_COMPLEX])
        text_size, _ = cv2.getTextSize(text, font, fontScale, thickness)
        text_width, text_height = text_size
        if(text_width < ((image.shape[0]/3) * 2)):
            check = False
    valid = False
    while(valid == False):
        image = image1
        x = random.randint(0, image.shape[0] - text_width - 5)
        y = random.randint(text_height + 5, image.shape[1])
        mask = image[y-text_height:y, x:x+text_width]
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        ret,mask = cv2.threshold(mask,127,255,cv2.THRESH_BINARY)
        white = np.sum(mask == 255)
        black = np.sum(mask == 0)
        if(white > black):
            color = (random.randint(0, 127), random.randint(0, 127), random.randint(0, 127))
        else:
            color = (random.randint(127, 255), random.randint(127, 255), random.randint(127, 255))
        org = (x, y)
        r_center = (x + (text_width/2), y - (text_height/2))
        overlay = np.zeros((image.shape[0],image.shape[1],3), np.uint8)
        overlay = cv2.putText(overlay, text, org, font, fontScale, color, thickness, cv2.LINE_AA, False)
        rotation_M = cv2.getRotationMatrix2D(org, rotation, 1)
        overlay = cv2.warpAffine(overlay, rotation_M, (overlay.shape[1], overlay.shape[0]))
        image = cv2.addWeighted(image, opacity, overlay, opacity,0, overlay)
        overlay = np.zeros((image.shape[0],image.shape[1],3), np.uint8)
        overlay = cv2.putText(overlay, text, org, font, fontScale, (255,255,255), thickness, cv2.LINE_AA, False)
        rotation_M = cv2.getRotationMatrix2D(org, rotation, 1)
        overlay = cv2.warpAffine(overlay, rotation_M, (overlay.shape[1], overlay.shape[0]))
        if(rotation < 0):
            xmin = x - 0.3*text_height
            ymin = y - text_height
            xmax = x + text_width*math.cos(math.radians(rotation*(-1))) + text_height
            ymax = y + 0.3*text_height + math.sin(math.radians(rotation*(-1)))*text_width
        else:
            xmin = x - 0.3*text_height
            ymin = y - text_height - math.sin(math.radians(rotation))*text_width
            xmax = x + math.cos(math.radians(rotation))*text_width
            ymax = y + 0.3*text_height
        cropped_text_mask = overlay[int(ymin):int(ymax), int(xmin):int(xmax)]
        cropped_text_image = image[int(ymin):int(ymax), int(xmin):int(xmax)]
        if(xmin >= 0 and xmax < image.shape[0] and ymin >= 0 and ymax < image.shape[1]):
            valid = True
    #yolo_annotation = get_yolo((x, y - text_height), (x + text_width, y), image.shape[0], image.shape[1])
    #yolo_annotation = save_annotations(image, [x, y - text_height, x + text_width, y])
    writer = Writer('train/fake_watermark/images/' + str(counter) + '.jpg', image.shape[0], image.shape[1])
    writer.addObject('watermark', xmin, ymin, xmax, ymax)
    writer.save('train/fake_watermark/images/' + str(counter) + '.xml')
    return image, cropped_text_mask, cropped_text_image

In [ ]:
counter = 0
files = os.listdir('train/no_watermark')
for file in files:
    if(counter == 1000):
        break
    image = cv2.imread('train/no_watermark/' + file)
    watermarked_image, watermarked_overlay, watermarked_text = add_watermark(image, counter)
    cv2.imwrite('train/fake_watermark/images/' + str(counter) + '.jpg', watermarked_image)
    #cv2.imwrite('train/fake_watermark/cropped_text_mask/' + str(counter) + '.jpg', watermarked_overlay)
    #cv2.imwrite('train/fake_watermark/cropped_text_image/' + str(counter) + '.jpg', watermarked_text)
    #print(annotation)
    #with open('train/fake_watermark/images/' + str(counter) + '.txt', 'w+') as f:
    #    f.writelines(annotation)
    counter+=1